In [1]:
%matplotlib inline
import os
import sys

import keras
import matplotlib.pyplot as plt

import numpy as np

from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, Add, Dropout, Concatenate, AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.datasets import cifar10
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import pandas as pd

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from sklearn.metrics import accuracy_score
import tensorflow as tf

import zipfile
import io
from PIL import Image
#from tqdm import tqdm
import math

from imgaug import augmenters as iaa

random_state = 42
THRESHOLD = 0.05

def resblock(x, filters=64, kernel_size=(3, 3), activation='relu'):
    x_ = Conv2D(filters, kernel_size, padding='same')(x)
    x_ = BatchNormalization()(x_)
    x_ = Conv2D(filters, kernel_size, padding='same')(x_)
    x = Add()([x_, x])
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    return x

def resblock2(x, filters=64, kernel_size=(3, 3), activation='relu'):
    x = Conv2D(filters, (1,1), padding='same')(x) 
    x_ = BatchNormalization()(x)
    x_ = Conv2D(filters, (1,1), padding='same')(x_)
    x_ = BatchNormalization()(x_)
    x_ = Activation(activation)(x_)
    x_ = Conv2D(filters, kernel_size, padding='same')(x_)
    x_ = BatchNormalization()(x_)
    x_ = Activation(activation)(x_)
    x_ = Conv2D(filters, (1,1), padding='same')(x_)
    x_ = BatchNormalization()(x_)
    x = Add()([x_, x])
    return x

def resnet2(x, filters=64, kernel_size=(3, 3), depth=10):
    for i in range(depth):
        x = resblock2(x, filters=filters)
    return x

def dense_block(x, input_channels, growth_rate, nb_blocks):
    n_channels = input_channels
    orig = x
    for i in range(nb_blocks):
        x = BatchNormalization()(x)
        x = LeakyReLU(0.1)(x) #Activation("relu")(x)
        x = Conv2D(128, (1, 1), kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.1)(x) #Activation("relu")(x)
        # フィルター数 = 成長度合
        x = Conv2D(growth_rate, (3, 3), padding="same", kernel_initializer='he_normal')(x)
        # origと結合
        x = Concatenate()([orig, x])
        n_channels += growth_rate
    return x, n_channels

def transition_layer(x, input_channels, compression_factor=0.5):
    n_channels = int(input_channels * compression_factor)
    x = Conv2D(n_channels, (1, 1))(x)
    x = AveragePooling2D((2, 2))(x)
    return x, n_channels

def f1(y_true, y_pred):
    #y_pred = K.round(y_pred)
    y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), THRESHOLD), K.floatx())
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    #y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), THRESHOLD), K.floatx())
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1-K.mean(f1)

def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        eps =1e-12
        y_pred = K.clip(y_pred, eps, 1.-eps)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
#        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

def darknet_block(x, input_channels):
    ch_hid = input_channels//2
    x = Conv2D(ch_hid, (1,1), padding='valid')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.1)(x)
    x = Conv2D(input_channels, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.1)(x)
    return x

def weighted_binary_crossentropy(y_true, y_pred):
    eps =1e-12
    y_pred = K.clip(y_pred, eps, 1.-eps)
    zero_weight= 0.2
    one_weight=0.8
    # Original binary crossentropy (see losses.py):
    # K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)

    # Calculate the binary crossentropy
    b_ce = K.binary_crossentropy(y_true, y_pred)

    # Apply the weights
    weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
    weighted_b_ce = weight_vector * b_ce

    # Return the mean error
    return K.mean(weighted_b_ce)


Using TensorFlow backend.


In [2]:
data = pd.read_csv("data/train.csv", engine='python')
data.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [3]:
#毎回データをgenerateする関数
def generateData(zippath, xs, ys,batch_size=128, imsize=128):
    n_batches = math.ceil(len(xs) / batch_size)
    while True:   
        for i in range(n_batches):
            start = i * batch_size
            end = (i + 1) * batch_size
            ft = True
            z = zipfile.ZipFile(zippath)
            for dat, category in zip(xs[start:end], ys[start:end].str.split(' ')):
                imgname_red = dat+'_red.png'
                img_red = np.array(Image.open( io.BytesIO(z.read(imgname_red)) ).resize((imsize, imsize))).astype('float32')
                img_red = img_red/np.max(img_red)
                imgname_blue = dat+'_blue.png'
                img_blue = np.array(Image.open( io.BytesIO(z.read(imgname_blue)) ).resize((imsize, imsize))).astype('float32')
                img_blue = img_blue/np.max(img_blue)
                imgname_green = dat+'_green.png'
                img_green = np.array(Image.open( io.BytesIO(z.read(imgname_green)) ).resize((imsize, imsize))).astype('float32')
                img_green = img_green/np.max(img_green)
                imgname_yellow = dat+'_yellow.png'
                img_yellow = np.array(Image.open( io.BytesIO(z.read(imgname_yellow)) ).resize((imsize, imsize))).astype('float32')
                img_yellow = img_yellow/np.max(img_yellow)
                #define as numpy 512 x 512 x 4
                image_merged = np.concatenate((img_red[:,:,np.newaxis],img_blue[:,:,np.newaxis],
                                               img_green[:,:,np.newaxis],img_yellow[:,:,np.newaxis]), axis=-1)
                                
                #get category
                y = np.zeros(28)
                for key in category:
                    y[int(key)] = 1
                    
                #append as one numpy (for saving)
                if ft:
                    image_all = image_merged[np.newaxis,:]
                    category_all = y[np.newaxis,:]
                    ft = False
                else:
                    image_all = np.append(image_all, image_merged[np.newaxis, :], axis=0)
                    category_all = np.append(category_all, y[np.newaxis, :], axis=0)
            z.close()
#            print(sys.getsizeof(image_all), sys.getsizeof(category_all), image_all.shape)
            yield [image_all, category_all]
    
    
#毎回データをgenerateする関数
def generateDatav2(zippath, xs, ys,batch_size=128):
    n_batches = math.ceil(len(xs) / batch_size)
    while True:   
        for i in range(n_batches):
            start = i * batch_size
            end = (i + 1) * batch_size
            ft = True
            z = zipfile.ZipFile(zippath)
            for dat, category in zip(xs[start:end], ys[start:end].str.split(' ')):
                imgname_green = dat+'_green.png'
                img_green = np.array(Image.open( io.BytesIO(z.read(imgname_green)) )).astype('float32')
                img_green = img_green/np.max(img_green)
                img_green = img_green.reshape(1, 512, 512, 1)
                imgname_red = dat+'_red.png'
                img_red = np.array(Image.open( io.BytesIO(z.read(imgname_red)) )).astype('float32')
                img_red = img_red/np.max(img_red)
                img_red = img_red.reshape(1, 512, 512, 1)
                imgname_blue = dat+'_blue.png'
                img_blue = np.array(Image.open( io.BytesIO(z.read(imgname_blue)) )).astype('float32')
                img_blue = img_blue/np.max(img_blue)
                img_blue = img_blue.reshape(1, 512, 512, 1)
                imgname_yellow = dat+'_yellow.png'
                img_yellow = np.array(Image.open( io.BytesIO(z.read(imgname_yellow)) )).astype('float32')
                img_yellow = img_yellow/np.max(img_yellow)
                img_yellow = img_yellow.reshape(1, 512, 512, 1)
                                
                #get category
                y = np.zeros(28)
                for key in category:
                    y[int(key)] = 1
                    
                #append as one numpy (for saving)
                if ft:
                    img_green_batch = img_green
                    img_red_batch = img_red
                    img_blue_batch = img_blue
                    img_yellow_batch = img_yellow
                    category_all = y[np.newaxis,:]
                    ft = False
                else:
                    img_green_batch = np.append(img_green_batch, img_green, axis=0)
                    img_red_batch = np.append(img_red_batch, img_red, axis=0)
                    img_blue_batch = np.append(img_blue_batch, img_blue, axis=0)
                    img_yellow_batch = np.append(img_yellow_batch, img_yellow, axis=0)
                    category_all = np.append(category_all, y[np.newaxis, :], axis=0)
            z.close()
#            print(sys.getsizeof(image_all), sys.getsizeof(category_all), image_all.shape)
            yield [img_green_batch, img_red_batch, img_blue_batch , img_yellow_batch], category_all
    
def generateDatav2aug(zippath, xs, ys,batch_size=128):
    def augment(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
            ])], random_order=True)
        image_aug = augment_img.augment_image(image)
        return image_aug

    n_batches = math.ceil(len(xs) / batch_size)
    while True:   
        for i in range(n_batches):
            start = i * batch_size
            end = (i + 1) * batch_size
            ft = True
            z = zipfile.ZipFile(zippath)
            for dat, category in zip(xs[start:end], ys[start:end].str.split(' ')):
                imgname_green = dat+'_green.png'
                img_green = np.array(Image.open( io.BytesIO(z.read(imgname_green)) )).astype('float32')
                img_green = img_green/np.max(img_green)
                img_green = img_green.reshape(512, 512, 1)
                imgname_red = dat+'_red.png'
                img_red = np.array(Image.open( io.BytesIO(z.read(imgname_red)) )).astype('float32')
                img_red = img_red/np.max(img_red)
                img_red = img_red.reshape(512, 512, 1)
                imgname_blue = dat+'_blue.png'
                img_blue = np.array(Image.open( io.BytesIO(z.read(imgname_blue)) )).astype('float32')
                img_blue = img_blue/np.max(img_blue)
                img_blue = img_blue.reshape(512, 512, 1)
                imgname_yellow = dat+'_yellow.png'
                img_yellow = np.array(Image.open( io.BytesIO(z.read(imgname_yellow)) )).astype('float32')
                img_yellow = img_yellow/np.max(img_yellow)
                img_yellow = img_yellow.reshape(512, 512, 1)
                
                img_green = augment(img_green).reshape(1, 512, 512, 1)
                img_red = augment(img_red).reshape(1, 512, 512, 1)
                img_blue = augment(img_blue).reshape(1, 512, 512, 1)
                img_yellow = augment(img_yellow).reshape(1, 512, 512, 1)
                
                #get category
                y = np.zeros(28)
                for key in category:
                    y[int(key)] = 1
                    
                #append as one numpy (for saving)
                if ft:
                    img_green_batch = img_green
                    img_red_batch = img_red
                    img_blue_batch = img_blue
                    img_yellow_batch = img_yellow
                    category_all = y[np.newaxis,:]
                    ft = False
                else:
                    img_green_batch = np.append(img_green_batch, img_green, axis=0)
                    img_red_batch = np.append(img_red_batch, img_red, axis=0)
                    img_blue_batch = np.append(img_blue_batch, img_blue, axis=0)
                    img_yellow_batch = np.append(img_yellow_batch, img_yellow, axis=0)
                    category_all = np.append(category_all, y[np.newaxis, :], axis=0)
            z.close()
#            print(sys.getsizeof(image_all), sys.getsizeof(category_all), image_all.shape)
            yield [img_green_batch, img_red_batch, img_blue_batch , img_yellow_batch], category_all

def generateDataaug(zippath, xs, ys,batch_size=256, imsize=128):
    def augment(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
            ])], random_order=True)
        image_aug = augment_img.augment_image(image)
        return image_aug
    
    n_batches = math.ceil(len(xs) / batch_size)
    while True:   
        for i in range(n_batches):
            start = i * batch_size
            end = (i + 1) * batch_size
            ft = True
            z = zipfile.ZipFile(zippath)
            for dat, category in zip(xs[start:end], ys[start:end].str.split(' ')):
                imgname_red = dat+'_red.png'
                img_red = np.array(Image.open( io.BytesIO(z.read(imgname_red)) ).resize((imsize, imsize))).astype('float32')
                img_red = img_red/np.max(img_red)
                imgname_blue = dat+'_blue.png'
                img_blue = np.array(Image.open( io.BytesIO(z.read(imgname_blue)) ).resize((imsize, imsize))).astype('float32')
                img_blue = img_blue/np.max(img_blue)
                imgname_green = dat+'_green.png'
                img_green = np.array(Image.open( io.BytesIO(z.read(imgname_green)) ).resize((imsize, imsize))).astype('float32')
                img_green = img_green/np.max(img_green)
                imgname_yellow = dat+'_yellow.png'
                img_yellow = np.array(Image.open( io.BytesIO(z.read(imgname_yellow)) ).resize((imsize, imsize))).astype('float32')
                img_yellow = img_yellow/np.max(img_yellow)
                #define as numpy 512 x 512 x 4
                image_merged = np.concatenate((img_red[:,:,np.newaxis],img_blue[:,:,np.newaxis],
                                               img_green[:,:,np.newaxis],img_yellow[:,:,np.newaxis]), axis=-1)
                
                #get category
                y = np.zeros(28)
                for key in category:
                    y[int(key)] = 1
                image_merged = augment(image_merged)
                #append as one numpy (for saving)
                if ft:
                    image_all = image_merged[np.newaxis,:]
                    category_all = y[np.newaxis,:]
                    ft = False
                else:
                    image_all = np.append(image_all, image_merged[np.newaxis, :], axis=0)
                    category_all = np.append(category_all, y[np.newaxis, :], axis=0)
            z.close()
#            print(sys.getsizeof(image_all), sys.getsizeof(category_all), image_all.shape)
            yield image_all, category_all

In [4]:
direc1 = 'models/11_l_checkpoint.h5'
direc1_cross_entropy = 'models/11_lf_checkpoint.h5'
direc2 = 'models/11_l_eval.txt'
direc_out = 'submissions/submission11_l.csv'

x_train = data['Id']
y_train = data['Target']
(x_train_mini, x_test_mini, y_train_mini, y_test_mini) = train_test_split(x_train, y_train, test_size=0.01, random_state=0)

activation = 'relu'
cb_es = EarlyStopping(monitor='val_loss', patience=30, verbose=1, mode='auto')
check_point = ModelCheckpoint(direc1, monitor='val_loss', 
                              verbose=1, save_best_only=True, 
                              mode='auto', period=1)

blocks=[2,4,8,4]
n=32
k=32

#network
xin = Input(shape=(128,128,4,))


x = Conv2D(n, (1,1), kernel_initializer='he_normal')(xin)
x, n = dense_block(x, n, k, blocks[0])
gap1 = GlobalAveragePooling2D()(x)
x, n = transition_layer(x, n)

x, n = dense_block(x, n, k, blocks[1])
gap2 = GlobalAveragePooling2D()(x)
x, n = transition_layer(x, n)

x, n = dense_block(x, n, k, blocks[2])
gap3 = GlobalAveragePooling2D()(x)
x, n = transition_layer(x, n)

x, n = dense_block(x, n, k, blocks[3])
gap4 = GlobalAveragePooling2D()(x)

x = Concatenate()([gap1, gap2, gap3, gap4])
x = Dense(256)(x)
x = LeakyReLU(0.1)(x)
x = Dropout(0.3)(x)
x = Dense(256)(x)
x = LeakyReLU(0.1)(x)
x = Dropout(0.3)(x)
#x = GlobalAveragePooling2D()(x)

y = Dense(28, activation='sigmoid')(x)

model = Model(inputs=xin, outputs=y)

In [5]:
#model = keras.models.load_model(direc1)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 4)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 32) 160         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 128, 128, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [26]:
batch_size = 16

train_gen = generateDatav2aug('data/train.zip', x_train_mini, y_train_mini, batch_size=batch_size)
valid_gen = generateDatav2('data/train.zip', x_test_mini, y_test_mini, batch_size=batch_size)

#model.load_weights(direc1_cross_entropy)

model.fit_generator(train_gen,
                    steps_per_epoch=math.ceil(len(x_train_mini) / batch_size), 
                    epochs=1000,  
                    validation_data = valid_gen,
                    validation_steps = math.ceil(len(x_test_mini) / batch_size),
                    callbacks=[cb_es, check_point], 
                    verbose=1,
                    max_queue_size=10)

model.save_weights(direc1_cross_entropy)

#model.compile(loss=[focal_loss(alpha=.25, gamma=2)], optimizer=adam, metrics=['accuracy', f1])
model.load_weights(direc1_cross_entropy)



Epoch 1/1000
  48/1923 [..............................] - ETA: 36:54 - loss: 0.1100 - acc: 0.9608 - f1: 0.2031

KeyboardInterrupt: 

In [ ]:
batch_size = 32

adam = keras.optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', f1])
#model.compile(loss=[focal_loss(alpha=.25, gamma=2)], optimizer=adam, metrics=['accuracy', f1])


train_gen = generateDataaug('data/train.zip', x_train_mini, y_train_mini, batch_size=batch_size)
valid_gen = generateData('data/train.zip', x_test_mini, y_test_mini, batch_size=batch_size)

model.fit_generator(train_gen,
                    steps_per_epoch=math.ceil(len(x_train_mini) / batch_size), 
                    epochs=1000,  
                    validation_data = valid_gen,
                    validation_steps = math.ceil(len(x_test_mini) / batch_size),
                    callbacks=[cb_es, check_point], 
                    verbose=1,
                    max_queue_size=10)


Epoch 1/1000
962/962 [==============================] - 1056s 1s/step - loss: 0.1546 - acc: 0.9487 - f1: 0.1329 - val_loss: 0.1485 - val_acc: 0.9482 - val_f1: 0.1430

Epoch 00001: val_loss improved from inf to 0.14853, saving model to models/11_l_checkpoint.h5
Epoch 2/1000
962/962 [==============================] - 1057s 1s/step - loss: 0.1489 - acc: 0.9498 - f1: 0.1454 - val_loss: 0.1432 - val_acc: 0.9520 - val_f1: 0.1500

Epoch 00002: val_loss improved from 0.14853 to 0.14325, saving model to models/11_l_checkpoint.h5
Epoch 3/1000
962/962 [==============================] - 1070s 1s/step - loss: 0.1445 - acc: 0.9507 - f1: 0.1564 - val_loss: 0.1404 - val_acc: 0.9517 - val_f1: 0.1655

Epoch 00003: val_loss improved from 0.14325 to 0.14036, saving model to models/11_l_checkpoint.h5
Epoch 4/1000
962/962 [==============================] - 1084s 1s/step - loss: 0.1391 - acc: 0.9519 - f1: 0.1676 - val_loss: 0.1346 - val_acc: 0.9522 - val_f1: 0.1727

Epoch 00004: val_loss improved from 0.1403

962/962 [==============================] - 1082s 1s/step - loss: 0.0957 - acc: 0.9653 - f1: 0.2709 - val_loss: 0.0973 - val_acc: 0.9659 - val_f1: 0.2758

Epoch 00035: val_loss improved from 0.09798 to 0.09729, saving model to models/11_l_checkpoint.h5
Epoch 36/1000
962/962 [==============================] - 1081s 1s/step - loss: 0.0956 - acc: 0.9654 - f1: 0.2710 - val_loss: 0.0996 - val_acc: 0.9639 - val_f1: 0.2635

Epoch 00036: val_loss did not improve from 0.09729
Epoch 37/1000
962/962 [==============================] - 1086s 1s/step - loss: 0.0951 - acc: 0.9657 - f1: 0.2721 - val_loss: 0.1012 - val_acc: 0.9646 - val_f1: 0.2648

Epoch 00037: val_loss did not improve from 0.09729
Epoch 38/1000
962/962 [==============================] - 1082s 1s/step - loss: 0.0946 - acc: 0.9656 - f1: 0.2727 - val_loss: 0.1033 - val_acc: 0.9641 - val_f1: 0.2465

Epoch 00038: val_loss did not improve from 0.09729
Epoch 39/1000
962/962 [==============================] - 1086s 1s/step - loss: 0.0940 - acc

In [6]:
#predict from submission.csv
data_sub = pd.read_csv("data/sample_submission.csv", engine='python')
data_sub.head()

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,0
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0


In [7]:
def gen_Data_prediction(zippath, xs, batch_size=128, imsize=128):
    n_batches = math.ceil(len(xs) / batch_size)
    while True:   
        for i in range(n_batches):
            start = i * batch_size
            end = (i + 1) * batch_size
            ft = True
            z = zipfile.ZipFile(zippath)
            for dat in xs[start:end]:
                imgname_red = dat+'_red.png'
                img_red = np.array(Image.open( io.BytesIO(z.read(imgname_red)) ).resize((imsize, imsize))).astype('float32')
                imgname_blue = dat+'_blue.png'
                img_blue = np.array(Image.open( io.BytesIO(z.read(imgname_blue)) ).resize((imsize, imsize))).astype('float32')
                imgname_green = dat+'_green.png'
                img_green = np.array(Image.open( io.BytesIO(z.read(imgname_green)) ).resize((imsize, imsize))).astype('float32')
                imgname_yellow = dat+'_yellow.png'
                img_yellow = np.array(Image.open( io.BytesIO(z.read(imgname_yellow)) ).resize((imsize, imsize))).astype('float32')
                #define as numpy 512 x 512 x 4
                image_merged = np.append(img_red[:,:,np.newaxis],img_blue[:,:,np.newaxis], axis=-1)
                image_merged = np.append(image_merged,img_green[:,:,np.newaxis], axis=-1)
                image_merged = np.append(image_merged,img_yellow[:,:,np.newaxis], axis=-1)
                image_merged = image_merged/255 #standardize
                #print(sys.getsizeof(image_merged))
                #append as one numpy (for saving)
                if ft:
                    image_all = image_merged[np.newaxis,:]
                    ft = False
                else:
                    image_all = np.append(image_all, image_merged[np.newaxis, :], axis=0)
            z.close()
            #print(sys.getsizeof(image_all), sys.getsizeof(category_all), image_all.shape)
            yield image_all
            
def gen_Data_predictionv2(zippath, xs, batch_size=128):
    n_batches = math.ceil(len(xs) / batch_size)
    while True:   
        for i in range(n_batches):
            start = i * batch_size
            end = (i + 1) * batch_size
            ft = True
            z = zipfile.ZipFile(zippath)
            for dat in xs[start:end]:
                imgname_green = dat+'_green.png'
                img_green = np.array(Image.open( io.BytesIO(z.read(imgname_green)) )).astype('float32')
                img_green = img_green/np.max(img_green)
                img_green = img_green.reshape(1, 512, 512, 1)
                imgname_red = dat+'_red.png'
                img_red = np.array(Image.open( io.BytesIO(z.read(imgname_red)) )).astype('float32')
                img_red = img_red/np.max(img_red)
                img_red = img_red.reshape(1, 512, 512, 1)
                imgname_blue = dat+'_blue.png'
                img_blue = np.array(Image.open( io.BytesIO(z.read(imgname_blue)) )).astype('float32')
                img_blue = img_blue/np.max(img_blue)
                img_blue = img_blue.reshape(1, 512, 512, 1)
                imgname_yellow = dat+'_yellow.png'
                img_yellow = np.array(Image.open( io.BytesIO(z.read(imgname_yellow)) )).astype('float32')
                img_yellow = img_yellow/np.max(img_yellow)
                img_yellow = img_yellow.reshape(1, 512, 512, 1)
                #append as one numpy (for saving)
                if ft:
                    img_green_batch = img_green
                    img_red_batch = img_red
                    img_blue_batch = img_blue
                    img_yellow_batch = img_yellow
                    ft = False
                else:
                    img_green_batch = np.append(img_green_batch, img_green, axis=0)
                    img_red_batch = np.append(img_red_batch, img_red, axis=0)
                    img_blue_batch = np.append(img_blue_batch, img_blue, axis=0)
                    img_yellow_batch = np.append(img_yellow_batch, img_yellow, axis=0)
            z.close()
#            print(sys.getsizeof(image_all), sys.getsizeof(category_all), image_all.shape)
            yield [img_green_batch, img_red_batch, img_blue_batch , img_yellow_batch]

In [ ]:
direc1 = 'models/11_l_checkpoint.h5'
direc1_cross_entropy = 'models/11_final_checkpoint.h5'
direc2 = 'models/11_l_eval.txt'
direc_out = 'submissions/submission11_l.csv'

x_train = data['Id']
y_train = data['Target']
(x_train_mini, x_test_mini, y_train_mini, y_test_mini) = train_test_split(x_train, y_train, test_size=0.01, random_state=123)

activation = 'relu'
cb_es = EarlyStopping(monitor='val_loss', patience=30, verbose=1, mode='auto')
check_point = ModelCheckpoint(direc1, monitor='val_loss', 
                              verbose=1, save_best_only=True, mode='auto', period=1)
ch = 16




y = Dense(28, activation='sigmoid')(x)


adam = keras.optimizers.Adam(lr=0.001)
model = Model(inputs=[xing, xinr, xinb, xiny], outputs=y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', f1])



In [8]:
batch_size = 64

x_test = data_sub['Id']

pred_gen = gen_Data_predictionv2('data/test.zip', x_test, batch_size=batch_size)

pred_y = model.predict_generator(pred_gen, steps=math.ceil(len(x_test) / batch_size), verbose=1)

i = 0
data_sub['Predicted'] = data_sub['Predicted'].astype(object)
for y_sub in pred_y:
    ft=True
    for ar in np.where(y_sub>0.1)[0]:
        #print(i," ",np.where(y_sub>0.5)[0])
        if ft:
            data_sub['Predicted'][i] = str(ar)
            ft = False
        else:
            data_sub['Predicted'][i] = data_sub['Predicted'][i] +' '+ str(ar)
    if ft:
        data_sub['Predicted'][i] = np.where(y_sub==np.max(y_sub))[0][0]
    i+=1    

data_sub.to_csv(direc_out, index=False)

183/183 [==============================] - 585s 3s/step


In [62]:
i = 0
data_sub['Predicted'] = data_sub['Predicted'].astype(object)
for y_sub in pred_y:
    ft=True
    for ar in np.where(y_sub>0.3)[0]:
        #print(i," ",np.where(y_sub>0.3)[0])
        if ft:
            data_sub['Predicted'][i] = str(ar)
            ft = False
        else:
            data_sub['Predicted'][i] = data_sub['Predicted'][i] +' '+ str(ar)
    if ft:
        data_sub['Predicted'][i] = np.where(y_sub==np.max(y_sub))[0][0]
    i+=1    

data_sub.to_csv(direc_out, index=False)

In [53]:
y_sub = pred_y[1]
np.where(y_sub>0.4)

(array([0], dtype=int64),)

In [52]:
y_sub

array([  4.70158070e-01,   6.25024885e-02,   4.75552678e-02,
         3.34178545e-02,   5.42743802e-02,   1.65147826e-01,
         8.96948669e-03,   1.80736054e-02,   3.88039371e-05,
         1.76359626e-05,   2.26985912e-06,   1.03038698e-02,
         5.95179107e-03,   8.25559534e-03,   2.11130977e-02,
         5.14525105e-04,   2.69827731e-02,   1.14835799e-02,
         2.89440583e-02,   7.02944696e-02,   1.48548046e-03,
         9.48961452e-02,   1.29110264e-02,   1.44278649e-02,
         1.74770271e-03,   2.56729752e-01,   2.10440643e-02,
         5.18633096e-05], dtype=float32)

In [63]:
data_sub

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,2 3
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 14 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0 25
5,00109f6a-bac8-11e8-b2b7-ac1f6b6435d0,0
6,001765de-bacd-11e8-b2b8-ac1f6b6435d0,0 25
7,0018641a-bac9-11e8-b2b8-ac1f6b6435d0,0 25
8,00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0
9,0026f154-bac6-11e8-b2b7-ac1f6b6435d0,0 25


In [ ]:
direc1 = 'models/01_checkpoint.h5'
direc2 = 'models/01_eval.txt'
direc_out = 'submissions/submission01_x.csv'

x_train = data['Id']
y_train = data['Target']
(x_train_mini, x_test_mini, y_train_mini, y_test_mini) = train_test_split(x_train, y_train, test_size=0.01, random_state=42)

#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range = 0.2,
    zoom_range = 0.2
)
datagen.fit(x_train_mini)

activation = 'relu'
batch_size = 16
cb_es = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
check_point = ModelCheckpoint(direc1, monitor='val_loss', 
                              verbose=1, save_best_only=True, 
                              save_weights_only=False, mode='auto', period=1)
epochs = 100
blocks=[2,4,8,4]
n=32
k=32

#network
xin = Input(shape=(512,512,4,))

x = Conv2D(4, (3,3), padding='same', activation=activation, 
           kernel_initializer='he_normal')(xin)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(8, (1,1), padding='same', activation=activation, 
           kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Conv2D(8, (3,3), padding='same', activation=activation, 
           kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(16, (1,1), padding='same', activation=activation, 
           kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Conv2D(16, (3,3), padding='same', activation=activation, 
           kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
'''
x = Conv2D(16, (1,1), padding='same', activation=activation, 
           kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Conv2D(16, (3,3), padding='same', activation=activation, 
           kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
'''

x = Conv2D(n, (1,1), kernel_initializer='he_normal')(x)
for i in range(len(blocks)):
    #Transition
    if i != 0:
        x, n = transition_layer(x, n)
    #DenseBlock
    x, n = dense_block(x, n, k, blocks[i])



x = GlobalAveragePooling2D()(x)

y = Dense(28, activation='sigmoid')(x)


adam = keras.optimizers.Adam(lr=0.001)
model = Model(inputs=xin, outputs=y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])



In [ ]:
direc1 = 'models/03_checkpoint.h5'
direc2 = 'models/03_eval.txt'
direc_out = 'submissions/submission03.csv'

x_train = data['Id']
y_train = data['Target']
(x_train_mini, x_test_mini, y_train_mini, y_test_mini) = train_test_split(x_train, y_train, test_size=0.01, random_state=42)

activation = 'relu'
batch_size = 4
cb_es = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
check_point = ModelCheckpoint(direc1, monitor='val_loss', 
                              verbose=1, save_best_only=True, 
                              save_weights_only=False, mode='auto', period=1)
epochs = 100

#network
xin = Input(shape=(512,512,4,))

x = resblock2(xin, filters=8, kernel_size=(3, 3), activation='relu')
x = MaxPooling2D(pool_size=(2, 2))(x)
x = resblock2(x, filters=16, kernel_size=(3, 3), activation='relu')
x = MaxPooling2D(pool_size=(2, 2))(x)
x = resblock2(x, filters=32, kernel_size=(3, 3), activation='relu')
x = MaxPooling2D(pool_size=(2, 2))(x)
x = resblock2(x, filters=64, kernel_size=(3, 3), activation='relu')
x = MaxPooling2D(pool_size=(2, 2))(x)
x = resblock2(x, filters=128, kernel_size=(3, 3), activation='relu')
x = MaxPooling2D(pool_size=(2, 2))(x)
x = GlobalAveragePooling2D()(x)

y = Dense(28, activation='sigmoid')(x)


adam = keras.optimizers.Adam(lr=0.001)
model = Model(inputs=xin, outputs=y)
model.compile(loss=[focal_loss(alpha=.25, gamma=2)], optimizer=adam, metrics=['accuracy', f1])



In [25]:
direc1 = 'models/05_checkpoint.h5'
direc2 = 'models/05_eval.txt'
direc_out = 'submissions/submission05_x.csv'

x_train = data['Id']
y_train = data['Target']
(x_train_mini, x_test_mini, y_train_mini, y_test_mini) = train_test_split(x_train, y_train, test_size=0.01, random_state=42)

activation = 'relu'
batch_size = 8
cb_es = EarlyStopping(monitor='val_loss', patience=30, verbose=1, mode='auto')
check_point = ModelCheckpoint(direc1, monitor='val_loss', 
                              verbose=1, save_best_only=True, 
                              save_weights_only=False, mode='auto', period=1)
epochs = 100
blocks=[2,4,8,4]
n=32
k=32

#network
xin = Input(shape=(512,512,4,))

x = Conv2D(8, (5,5), padding='same', activation=activation, 
           kernel_initializer='he_normal', name='conv1')(xin)
x = BatchNormalization()(x)
xt = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(16, (3,3), padding='same', activation=activation, 
           kernel_initializer='he_normal', name='conv2')(x)
x = BatchNormalization()(xt)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(32, (3,3), padding='same', activation=activation, 
           kernel_initializer='he_normal', name='conv3')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

#x = Conv2D(16, (1,1), padding='same', activation=activation, 
#           kernel_initializer='he_normal')(x)
#x = BatchNormalization()(x)
#x = Conv2D(16, (3,3), padding='same', activation=activation, 
#           kernel_initializer='he_normal')(x)
#x = BatchNormalization()(x)
#x = MaxPooling2D(pool_size=(2, 2))(x)


x = Conv2D(n, (1,1), kernel_initializer='he_normal')(x)
for i in range(len(blocks)):
    #Transition
    if i != 0:
        x, n = transition_layer(x, n)
    #DenseBlock
    x, n = dense_block(x, n, k, blocks[i])



x = GlobalAveragePooling2D()(x)

y = Dense(28, activation='sigmoid')(x)


adam = keras.optimizers.Adam(lr=0.001)
model = Model(inputs=xin, outputs=y)
#model.compile(loss=[focal_loss(alpha=.25, gamma=2)], optimizer=adam, metrics=['accuracy', f1])
#model.compile(loss=[f1_loss], optimizer=adam, metrics=['accuracy', f1])

model =  keras.models.load_model(direc1, custom_objects={'f1_loss': f1_loss, 'f1': f1})
K.set_value(model.optimizer.lr, 0.0005)

print('compiled!')

compiled!


In [7]:
direc1 = 'models/06_checkpoint.h5'
direc2 = 'models/06_eval.txt'
direc_out = 'submissions/submission06.csv'

x_train = data['Id']
y_train = data['Target']
(x_train_mini, x_test_mini, y_train_mini, y_test_mini) = train_test_split(x_train, y_train, test_size=0.01, random_state=42)

activation = 'relu'
cb_es = EarlyStopping(monitor='val_loss', patience=30, verbose=1, mode='auto')
check_point = ModelCheckpoint(direc1, monitor='val_loss', 
                              verbose=1, save_best_only=True, 
                              save_weights_only=False, mode='auto', period=1)
epochs = 100

#network
encoder = keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=(512,512,3), pooling='avg')

xin = Input(shape=(512,512,4,))
x = BatchNormalization()(xin)
x = Conv2D(3, kernel_size=(1,1), activation='relu')(xin)
x = BatchNormalization()(x)
x = encoder(x)
#x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
y = Dense(28, activation='sigmoid')(x)


adam = keras.optimizers.Adam(lr=0.001)
model = Model(inputs=xin, outputs=y)
model.compile(loss=[focal_loss(alpha=.25, gamma=2)], optimizer=adam, metrics=['accuracy', f1])
#model.compile(loss=[f1_loss], optimizer=adam, metrics=['accuracy', f1])

#model =  keras.models.load_model(direc1, custom_objects={'f1_loss': f1_loss, 'f1': f1})
#K.set_value(model.optimizer.lr, 0.0005)

print('compiled!')

compiled!


In [14]:
direc1 = 'models/07_checkpoint.h5'
direc2 = 'models/07_eval.txt'
direc_out = 'submissions/submission07.csv'

load_weight = False

x_train = data['Id']
y_train = data['Target']
(x_train_mini, x_test_mini, y_train_mini, y_test_mini) = train_test_split(x_train, y_train, test_size=0.01, random_state=42)

activation = 'relu'
cb_es = EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='auto')
check_point = ModelCheckpoint(direc1, monitor='val_loss', 
                              verbose=1, save_best_only=True, 
                              save_weights_only=True, mode='auto', period=1)
epochs = 1000
blocks=[2,4,8,4]
n=32
k=32

#network
xin = Input(shape=(512,512,4,))

x = Conv2D(8, (3,3), strides=(2,2), padding='same',  
           kernel_initializer='he_normal')(xin) #512x512x4 -> 256x256x8
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)

x = Conv2D(16, (3,3), strides=(2,2), padding='same', 
           kernel_initializer='he_normal')(x) # 256x256x8 -> 128x128x16
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)

x = Conv2D(n, (1,1), kernel_initializer='he_normal')(x)
for i in range(len(blocks)):
    #Transition
    if i != 0:
        x, n = transition_layer(x, n)
    #DenseBlock
    x, n = dense_block(x, n, k, blocks[i])

x = GlobalAveragePooling2D()(x)
x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)
y = Dense(28, activation='sigmoid')(x)


adam = keras.optimizers.Adam(lr=0.001)
model = Model(inputs=xin, outputs=y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', f1])
#model.compile(loss=[focal_loss(alpha=.25, gamma=2)], optimizer=adam, metrics=['accuracy', f1])

if load_weight:
    model.load_weights(direc1)



In [24]:
direc1 = 'models/08_checkpoint.h5'
direc2 = 'models/08_eval.txt'
direc_out = 'submissions/submission08.csv'

load_weight = False

x_train = data['Id']
y_train = data['Target']
(x_train_mini, x_test_mini, y_train_mini, y_test_mini) = train_test_split(x_train, y_train, test_size=0.01, random_state=42)

activation = 'relu'
cb_es = EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='auto')
check_point = ModelCheckpoint(direc1, monitor='val_loss', 
                              verbose=1, save_best_only=True, 
                              save_weights_only=True, mode='auto', period=1)
epochs = 1000
blocks=[2,4,8,4]
n=32
k=32

#network
xin = Input(shape=(512,512,4,))

x = Conv2D(8, (3,3), strides=(2,2), padding='same',  
           kernel_initializer='he_normal')(xin) #512x512x4 -> 256x256x8
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)
x = Dropout(0.5)(x)

x = Conv2D(16, (3,3), strides=(2,2), padding='same', 
           kernel_initializer='he_normal')(x) # 256x256x8 -> 128x128x16
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)
x = Dropout(0.5)(x)

x = Conv2D(n, (1,1), kernel_initializer='he_normal')(x)
for i in range(len(blocks)):
    #Transition
    if i != 0:
        x, n = transition_layer(x, n)
    #DenseBlock
    x, n = dense_block(x, n, k, blocks[i])

x = GlobalAveragePooling2D()(x)
x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)
x = Dropout(0.5)(x)
y = Dense(28, activation='sigmoid')(x)


adam = keras.optimizers.Adam(lr=0.001)
model = Model(inputs=xin, outputs=y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', f1])
#model.compile(loss=[focal_loss(alpha=.25, gamma=2)], optimizer=adam, metrics=['accuracy', f1])

if load_weight:
    model.load_weights(direc1)



In [15]:
direc1 = 'models/09_l_checkpoint.h5'
direc2 = 'models/09_l_eval.txt'
direc_out = 'submissions/submission09_l.csv'

load_weight = True

x_train = data['Id']
y_train = data['Target']
(x_train_mini, x_test_mini, y_train_mini, y_test_mini) = train_test_split(x_train, y_train, test_size=0.01, random_state=42)

activation = 'relu'
cb_es = EarlyStopping(monitor='val_loss', patience=30, verbose=1, mode='auto')
check_point = ModelCheckpoint(direc1, monitor='val_loss', 
                              verbose=1, save_best_only=True, 
                              save_weights_only=True, mode='auto', period=1)
epochs = 1000


#network
xin = Input(shape=(512,512,4,))

x = Conv2D(32, (5,5), padding='same')(xin)
#x = Conv2D(32, (3,3), strides=(2,2), padding='same')(xin)
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)

x = Conv2D(64, (3,3), strides=(2,2), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)

x = darknet_block(x, 64, LReLu_rate=0.1)
x = darknet_block(x, 64, LReLu_rate=0.1)
x = Conv2D(128, (3,3), strides=(2,2), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)

x = darknet_block(x, 128, LReLu_rate=0.1)
x = darknet_block(x, 128, LReLu_rate=0.1)
x = Conv2D(256, (3,3), strides=(2,2), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)

x = darknet_block(x, 256, LReLu_rate=0.1)
x = darknet_block(x, 256, LReLu_rate=0.1)
x = Conv2D(512, (3,3), strides=(2,2), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)

x = darknet_block(x, 512, LReLu_rate=0.1)
x = darknet_block(x, 512, LReLu_rate=0.1)
x = Conv2D(1024, (3,3), strides=(2,2), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)

x = GlobalAveragePooling2D()(x)
y = Dense(28, activation='sigmoid')(x)


adam = keras.optimizers.Adam(lr=0.001)
model = Model(inputs=xin, outputs=y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', f1])
#model.compile(loss=[focal_loss(alpha=.25, gamma=2)], optimizer=adam, metrics=['accuracy', f1])

if load_weight:
    model.load_weights(direc1)



In [4]:
direc1 = 'models/10_l_checkpoint.h5'
direc1_cross_entropy = 'models/10_cs_checkpoint.h5'
direc2 = 'models/10_l_eval.txt'
direc_out = 'submissions/submission10_l.csv'

x_train = data['Id']
y_train = data['Target']
(x_train_mini, x_test_mini, y_train_mini, y_test_mini) = train_test_split(x_train, y_train, test_size=0.01, random_state=42)

activation = 'relu'
cb_es = EarlyStopping(monitor='val_loss', patience=30, verbose=1, mode='auto')
check_point = ModelCheckpoint(direc1, monitor='val_loss', 
                              verbose=1, save_best_only=True, 
                              save_weights_only=True, mode='auto', period=1)
ch = 16

def base_network(xin):
    x = Conv2D(ch, (5,5), strides=(2,2), padding='same')(xin)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.1)(x)

    x = darknet_block(x, ch, LReLu_rate=0.1)
    x = darknet_block(x, ch, LReLu_rate=0.1)
    x = Conv2D(ch*2, (3,3), strides=(2,2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.1)(x)

    x = darknet_block(x, ch*2, LReLu_rate=0.1)
    x = darknet_block(x, ch*2, LReLu_rate=0.1)
    x = Conv2D(ch*4, (3,3), strides=(2,2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.1)(x)

    x = darknet_block(x, ch*4, LReLu_rate=0.1)
    x = darknet_block(x, ch*4, LReLu_rate=0.1)
    x = Conv2D(ch*8, (3,3), strides=(2,2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.1)(x)

    x = GlobalAveragePooling2D()(x)
    return x

#network1
xing = Input(shape=(512,512,1,))
xg = base_network(xing)

#network2
xinr = Input(shape=(512,512,1,))
xr = base_network(xinr)

#network3
xinb = Input(shape=(512,512,1,))
xb = base_network(xinb)

#network4
xiny = Input(shape=(512,512,1,))
xy = base_network(xiny)

x = Concatenate()([xg,xr,xb,xy])

y = Dense(28, activation='sigmoid')(x)


adam = keras.optimizers.Adam(lr=0.0005)
model = Model(inputs=[xing, xinr, xinb, xiny], outputs=y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', f1])
#model.compile(loss=[focal_loss(alpha=.25, gamma=2)], optimizer=adam, metrics=['accuracy', f1])

model.load_weights(direc1)

